# DiaC_MarEDAT_python3

In [ ]:
class DiaC_MarEDAT_comp:
    '''
    class DiaC_MarEDAT_comp(self,runname,resultpath,savepath,meshpath,ncfileMaredat,first_year,last_year,
                 mapproj='pc',verbose = False,
                 savefig=False)
                 
    '''
    
    def __init__(self,runname,resultpath,savepath,meshpath,ncfileMaredat,first_year,last_year,
                 mapproj='pc', verbose = False,
                 savefig=False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.verbose = verbose
        self.savefig = savefig
        self.ncfileMaredat = ncfileMaredat
        

        import pyfesom2 as pf
        import math
        import numpy as np
        import matplotlib.pylab as plt
        import matplotlib.cm as cm
        from matplotlib import gridspec
        from netCDF4 import Dataset
        import time
        from datetime import date
        import cartopy.crs as ccrs
        import pangaeapy.pandataset as pd

        import skill_metrics as sm
        from plot_Taylor_normalized import plt_Taylor_norm

        !jupyter nbconvert --to=python F2R_sensitivity_toolbox.ipynb
        import F2R_sensitivity_toolbox as stx

        import warnings
        warnings.filterwarnings('ignore')
                    
        #ncfileMaredat = "/home/ollie/mozeisin/obs/MarEDat/MarEDat20120716Diatoms.nc"
        
        years = [self.fyear, self.lyear]
        self.years = years

        # mesh initialization -----------------------------------------------------------
        mesh = pf.load_mesh(self.meshpath)

        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.meshpath +'fesom.mesh.diag.nc'
        #!ncdump -h $meshdiag

        diag = pf.get_meshdiag(mesh)
        #print(diag)
        #print(diag['Z']) # depth of layers
        mesh_depths = diag['Z'].values

        if self.verbose:
            stx.check_plot_mesh(mesh, plot_globe = False)
        
        # load Maredat -------------------------------------------------------------------------------
        lat_maredat, lon_maredat, maredat_layered = stx.maredat_load(ncfileMaredat)
        
        # maredat mean min max of 0-5m
        print('MarEDAT mean min max at 0-5m:')
        print(np.nanmean(maredat_layered[0]), np.nanmin(maredat_layered[0]), np.nanmax(maredat_layered[0]))
        
        # load FESOM DiaC data -------------------------------------------------------------------------------
        if self.verbose:
            NCfesom = self.resultpath + '/DiaC.'+self.runname+'.1948.nc' # DiaC:units = "[mmol/m3]"
            !ncdump -h $NCfesom

        units = 'Diatom intracell. Carbon [mg C m$^{{-3}}$]\n{0}-{1}'.format(self.years[0],self.years[1])
        #units = 'Intracellular C [mg C m$^{-3}$]'
        
        #DiaC = pf.get_data(self.resultpath, "DiaC", years, mesh, runid=self.runname, how="mean", compute=True, depth=depth)
        #DiaC = DiaC * 12.01

        model = stx.fesom_to_maredat_levels(self.resultpath, self.runname, self.years, mesh, lon_maredat, lat_maredat)

        # plot Maredat and FESOM DiaC data -------------------------------------------------------------------------------        
        if self.verbose:
            # label used for plotting
            figlabel = '' #'FESOM DiaC' 
            # extra label used for saving output
            savelabel = ''

            stx.plot_2cols(model,figlabel,savelabel, maredat_layered, lon_maredat, lat_maredat, self.mapproj, self.years, self.savefig, self.savepath)

        # prepare for statistics -------------------------------------------------------------------------------        
        # mask model where MarEDAT not available
        model_ma = stx.mask_model_with_maredat(model,maredat_layered)

        if self.verbose: # control masking
            fig, axes = plt.subplots(1,2, 
                                     subplot_kw=dict(projection=ccrs.Robinson()),
                                     gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                                     figsize=(20,20))
            # 0-5 m maredat 
            m1 = axes[0]
            f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[0]), 
                               transform = ccrs.PlateCarree(),
                                shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                                cmap=plt.cm.viridis)
            stx.mygrid(m1)
            m1.set_title('MarEDAT 0-5 m', fontsize=20)

            # FESOM 
            m1 = axes[1]
            f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(model_ma[0]), 
                               transform = ccrs.PlateCarree(),
                                shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                                cmap=plt.cm.viridis)
            stx.mygrid(m1)
            m1.set_title('FESOM 0-5 m', fontsize=20)
            
        # Taylor statistics ---------------------------------------------------------------------------------
        depth_range = [0,5,25,100,'bot']
        
        if self.verbose:
            fig, sdev, crmsd, ccoef_model = stx.plot_Taylor_all_depths(maredat_layered,model_ma,depth_range,plot=True)
            stx.plot_scatter(model_ma, figlabel, savelabel, maredat_layered, self.years, self.savefig, self.savepath, ccoef_model)
            
        else:
            fig, sdev, crmsd, ccoef_model = stx.plot_Taylor_all_depths(maredat_layered,model_ma,depth_range,plot=False)

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo12'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/mo12/'

    htmlname     =  runid + '.html'
    htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1970
    last_year  = 1980

    # input data ------------------------------------------------------------------------------------
    ncfileMaredat = "/home/ollie/mozeisin/obs/MarEDat/MarEDat20120716Diatoms.nc"

        
    # now test ------------------------------------------------------------------------------------
    test = DiaC_MarEDAT_comp(runid,resultpath,savepath,meshpath,ncfileMaredat,first_year,last_year,
                 mapproj='pc',
                verbose = False,
                 savefig=False,
                 layerwise=False)
                
